In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image


In [2]:
import torch
import torch.nn as nn
import torch.distributed as dist

class GatherLayer(torch.autograd.Function):
    """Gather tensors from all process, supporting backward propagation."""

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        output = [torch.zeros_like(input) for _ in range(dist.get_world_size())]
        dist.all_gather(output, input)
        return tuple(output)

    @staticmethod
    def backward(ctx, *grads):
        (input,) = ctx.saved_tensors
        grad_out = torch.zeros_like(input)
        grad_out[:] = grads[dist.get_rank()]
        return grad_out

#
#class NT_Xent(nn.Module):
#    def __init__(self, batch_size, temperature, world_size):
#        super(NT_Xent, self).__init__()
#        self.batch_size = batch_size
#        self.temperature = temperature
#        self.world_size = world_size
#
#        self.mask = self.mask_correlated_samples(batch_size, world_size)
#        self.criterion = nn.CrossEntropyLoss(reduction="sum")
#        self.similarity_f = nn.CosineSimilarity(dim=2)
#
#    def mask_correlated_samples(self, batch_size, world_size):
#        N = 2 * batch_size * world_size
#        mask = torch.ones((N, N), dtype=bool)
#        mask = mask.fill_diagonal_(0)
#        for i in range(batch_size * world_size):
#            mask[i, batch_size * world_size + i] = 0
#            mask[batch_size * world_size + i, i] = 0
#        return mask
#
#    def forward(self, z_i, z_j):
#        """
#        We do not sample negative examples explicitly.
#        Instead, given a positive pair, similar to (Chen et al., 2017), we treat the other 2(N − 1) augmented examples within a minibatch as negative examples.
#        """
#        N = 2 * self.batch_size * self.world_size
#
#        if self.world_size > 1:
#            z_i = torch.cat(GatherLayer.apply(z_i), dim=0)
#            z_j = torch.cat(GatherLayer.apply(z_j), dim=0)
#        z = torch.cat((z_i, z_j), dim=0)
#
#        sim = self.similarity_f(z.unsqueeze(1), z.unsqueeze(0)) / self.temperature
#
#        sim_i_j = torch.diag(sim, self.batch_size * self.world_size)
#        sim_j_i = torch.diag(sim, -self.batch_size * self.world_size)
#
#        # We have 2N samples, but with Distributed training every GPU gets N examples too, resulting in: 2xNxN
#        positive_samples = torch.cat((sim_i_j, sim_j_i), dim=0).reshape(N, 1)
#        negative_samples = sim[self.mask].reshape(N, -1)
#
#        labels = torch.zeros(N).to(positive_samples.device).long()
#        logits = torch.cat((positive_samples, negative_samples), dim=1)
#        loss = self.criterion(logits, labels)
#        loss /= N
#        return loss

In [3]:

class NT_Xent(nn.Module):
    def __init__(self, batch_size, temperature, world_size):
        super(NT_Xent, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.world_size = world_size

        #self.mask = self.mask_correlated_samples(batch_size, world_size)
        self.criterion = nn.CrossEntropyLoss(reduction="sum")
        self.similarity_f = nn.CosineSimilarity(dim=2)

    def mask_correlated_samples(self, batch_size, world_size):
        N = 2 * batch_size * world_size
        mask = torch.ones((N, N), dtype=bool)
        mask = mask.fill_diagonal_(0)
        for i in range(batch_size * world_size):
            mask[i, batch_size * world_size + i] = 0
            mask[batch_size * world_size + i, i] = 0
        return mask

    def forward(self, z_i, z_j):
        """
        We do not sample negative examples explicitly.
        Instead, given a positive pair, similar to (Chen et al., 2017), we treat the other 2(N − 1) augmented examples within a minibatch as negative examples.
        """


        # Filter out all-zero samples from z_i and z_j
        #non_zero_mask_i = torch.any(z_i != 0, dim=1)
        #non_zero_mask_j = torch.any(z_j != 0, dim=1)

        all_zero_mask = torch.all(z_i == 0, dim=1) & torch.all(z_j == 0, dim=1)

        # Invert the mask to keep rows where at least one of z_i or z_j is non-zero
        keep_mask = ~all_zero_mask

        # Only keep non-zero samples in z_i and z_j
        z_i_non_zero = z_i[keep_mask]
        z_j_non_zero = z_j[keep_mask]


        N = 2 * len(z_i_non_zero) * self.world_size

        # Concatenate non-zero samples
        z = torch.cat((z_i_non_zero, z_j_non_zero), dim=0)

        #z = torch.cat((z_i, z_j), dim=0)
        if self.world_size > 1:
            z = torch.cat(GatherLayer.apply(z), dim=0)

        sim = self.similarity_f(z.unsqueeze(1), z.unsqueeze(0)) / self.temperature

        sim_i_j = torch.diag(sim, len(z_i_non_zero) * self.world_size)
        sim_j_i = torch.diag(sim, -1 * len(z_i_non_zero) * self.world_size)

        # We have 2N samples, but with Distributed training every GPU gets N examples too, resulting in: 2xNxN
        positive_samples = torch.cat((sim_i_j, sim_j_i), dim=0).reshape(N, 1)
        self.mask = self.mask_correlated_samples(len(z_i_non_zero), self.world_size)
        negative_samples = sim[self.mask].reshape(N, -1)

        labels = torch.zeros(N).to(positive_samples.device).long()
        logits = torch.cat((positive_samples, negative_samples), dim=1)
        loss = self.criterion(logits, labels)
        loss /= N
        return loss

In [4]:
# prompt: get number of images in a folder
import os
num_images1 = len(os.listdir('/content/drive/MyDrive/data2/train/0_N'))
print(num_images1)

num_images1 = len(os.listdir('/content/drive/MyDrive/data2/train/2_UDH'))
print(num_images1)

num_images1 = len(os.listdir('/content/drive/MyDrive/data2/train/3_FEA'))
print(num_images1)

num_images1 = len(os.listdir('/content/drive/MyDrive/data2/train/4_ADH'))
print(num_images1)

num_images1 = len(os.listdir('/content/drive/MyDrive/data2/train/5_DCIS'))
print(num_images1)

num_images1 = len(os.listdir('/content/drive/MyDrive/data2/train/6_IC'))
print(num_images1)


180
105
115
292
317
937


In [5]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

def read_images_from_folder(folder_path, images, labels, label):



    # List all files in the folder
    files = os.listdir(folder_path)

    # Ensure that the list of files is not empty
    if files:
        # Sort the files to ensure consistent order
        files.sort()

        # Iterate over the files and read images
        for file_name in files:
            file_path = os.path.join(folder_path, file_name)

            # Open the image using PIL
            image = Image.open(file_path)

            # Convert the PIL image to a numpy array
            image_array = np.array(image)

            # Append the image array to the list
            images.append(image_array)
            labels.append(label)

    return images, labels

# Example usage
train_images = []
train_labels = []
test_images = []
test_labels = []

labels_list = ['0_N', '2_UDH', '3_FEA', '4_ADH', '5_DCIS', '6_IC']

train_folder_path = "/content/drive/MyDrive/data2/train/"  # Replace with the actual path to your folder
test_folder_path = "/content/drive/MyDrive/data2/test/"

for label in labels_list:
  train_images, train_labels = read_images_from_folder(f"{train_folder_path}{label}", train_images, train_labels, label)
  test_images, test_labels = read_images_from_folder(f"{test_folder_path}{label}", test_images, test_labels, label)


# Display every image in the list
#for i, image_array in enumerate(images):
#    plt.imshow(image_array)
#    plt.title(f"Image {i + 1}")
#    plt.show()


In [6]:
len(train_images)

1946

In [7]:
len(train_labels)

1946

In [8]:
len(test_images)

512

In [9]:
len(test_labels)

512

In [ ]:
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image
import random

class CustomDataset(data.Dataset):
    def __init__(self, images, labels, transform=None, overlap_percentage=0.25):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.overlap_percentage = overlap_percentage

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        label_to_index = {'0_N':0, '2_UDH':1, '3_FEA':2, '4_ADH':3, '5_DCIS':4, '6_IC':5}


        # Convert numpy array to PIL Image
        image = Image.fromarray(image)

        if self.transform:
            patch_size = 224  # Set your desired patch size

            # Calculate center coordinates
            center_x = image.width // 2
            center_y = image.height // 2

            # Calculate the overlap dimensions
            overlap_x = int(patch_size * self.overlap_percentage)
            overlap_y = int(patch_size * self.overlap_percentage)

            # Randomly choose offsets for the second patch within the overlap area
            offset_x = random.randint(-overlap_x, overlap_x)
            offset_y = random.randint(-overlap_y, overlap_y)

            # Extract patches from the center with overlap
            patch1 = image.crop((center_x - patch_size // 2, center_y - patch_size // 2,
                                 center_x + patch_size // 2, center_y + patch_size // 2))

            patch2 = image.crop((center_x - patch_size // 2 + offset_x, center_y - patch_size // 2 + offset_y,
                                 center_x + patch_size // 2 + offset_x, center_y + patch_size // 2 + offset_y))

            # Apply the common transformations
            image_transformed1 = self.transform(patch1)
            image_transformed2 = self.transform(patch2)

        return image_transformed1, image_transformed2, label

    def __len__(self):
        return len(self.images)


In [ ]:

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.RandomVerticalFlip(),  # Randomly flip the image vertically
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Random perspective transformation
    transforms.RandomRotation(degrees=(-45, 45)),  # Randomly rotate the image within the range of -45 to 45 degrees
    transforms.ToTensor(),  # Converts the image to a PyTorch tensor
])


In [ ]:

# Create an instance of CustomDataset
train_dataset = CustomDataset(train_images, train_labels, transform=transform)

#test_dataset = CustomDataset(test_images, test_labels, transform=transform)

# Create a DataLoader
batch_size = 16  # Set your desired batch size
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [10]:
# prompt: define a CNN model with backbone of resnet18 and reduc the last layer of resnet 18 20 256 and add a header to it  that has 3 layers

import torch
import torch.nn as nn
from torchvision import models


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # Use the pretrained ResNet18 model as the backbone
        self.backbone = models.resnet18(pretrained=True)

        # Reduce the last layer of ResNet18 to 256 outputs
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 256)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


        # Define the header with 3 layers
        self.header = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),  # Adjust the output size based on your classification task
        )

    def forward(self, x):
        # Check the number of input channels
        #num_input_channels = x.size(1)

        # Adjust the first convolutional layer based on the number of input channels
        #self.backbone.conv1 = nn.Conv2d(num_input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # Extract features from the backbone
        features = self.backbone(x)

        # Global average pooling
        #features = nn.functional.adaptive_avg_pool2d(features, (1, 1))
        #features = features.view(features.size(0), -1)

        # Pass the features through the header
        output = self.header(features)

        # Apply softmax activation
        #output = self.softmax(output)

        return output



In [ ]:

# Create an instance of the model
model = MyModel()

In [ ]:
# Create an instance of the model
#model = SimpleCNN()

# Define loss function and optimizer

#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# prompt: define device and use gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [ ]:
# Check the device of the model's parameters
for name, param in model.named_parameters():
    print(f"Parameter {name} on device: {param.device}")

In [ ]:
device

device(type='cuda')

In [ ]:
temperature = .5
contrastive_loss = NT_Xent(batch_size, temperature, world_size=1)
#inter_cxr_contrastive = contrastive_loss(inter_cxr_outputs, inter_cxr_outputs_aug)


In [ ]:
# prompt: train the model for 10 epochs and print training loss

# Train the model for 10 epochs
num_epochs = 15

for epoch in range(num_epochs):
    running_loss = 0.0

    # Iterate over the training data
    for i, data in enumerate(train_dataloader):
        # Get the inputs
        inputs1, inputs2, labels = data

        # Move data to the device
        inputs1 = inputs1.to(device)
        inputs2 = inputs2.to(device)
        #labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()
        #print(inputs1.device)
        #print(model.device)

        # Forward pass
        outputs1 = model(inputs1)
        outputs2 = model(inputs2)
        #print(len(outputs1))
        #print(len(outputs2))
        if len(outputs1) == 16 and len(outputs2) == 16:
          loss = contrastive_loss(outputs1, outputs2)

          # Backward pass
          loss.backward()

          # Update the parameters
          optimizer.step()

          # Print statistics
          running_loss += loss.item()
          if i % 100 == 0:  # Print every 2000 mini-batches
              print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}")
              running_loss = 0.0

print("Finished Training")


In [ ]:
# prompt: save the model after training save it in given path

torch.save(model.state_dict(), "/content/drive/MyDrive/model_state.pt")


PART 2 OF PRETRAINING THE MODEL !!:

In [ ]:
!pip install staintools

In [12]:
!pip install spams

In [13]:
from staintools import read_image, StainNormalizer

In [14]:
!pip install opencv-python


In [15]:
import cv2
from staintools import StainNormalizer

In [16]:
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image
import random
import copy

class CustomDataset(data.Dataset):
    def __init__(self, images, labels, transform=None, overlap_percentage=0.25):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.overlap_percentage = overlap_percentage

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        label_to_index = {'0_N':0, '2_UDH':1, '3_FEA':2, '4_ADH':3, '5_DCIS':4, '6_IC':5}


        # Convert numpy array to PIL Image
        image = Image.fromarray(image)
        bad_list = [1246, 1392]
        random_number = 964
        #while(True):
        #  random_number = random.randint(0, len(self.images) - 2)
        #  print("kiiiiiiiirrrrrrrkhaaaaaaaarrrrrrrr")
        #  print(random_number)
        #  if random_number not in bad_list:
        #    break


        #for i in range(len(images)):
        opencv_image = cv2.cvtColor(self.images[index], cv2.COLOR_RGB2BGR)
        im_index = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)

        opencv_image = cv2.cvtColor(self.images[random_number], cv2.COLOR_RGB2BGR)
        im_base = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)
        try:

          normalizer = StainNormalizer(method='macenko')
          normalizer.fit(im_base)
          macenko_image = normalizer.transform(im_index)

          # Your existing code
          normalizer = StainNormalizer(method='vahadane')
          normalizer.fit(im_base)
          vahadane_image = normalizer.transform(im_index)
        except:
          macenko_image = copy.deepcopy(im_index)
          vahadane_image = copy.deepcopy(im_index)


        if self.transform:


            # Apply the common transformations
            #print(type(macenko_image))
            #print("####")
            #print("original")
            #print(self.images[index].size)
            #print("numpy size")
            #print(macenko_image.size)
            #print(vahadane_image.size)
            macenko_image = Image.fromarray(macenko_image)
            vahadane_image = Image.fromarray(vahadane_image)
            #print("numpy size")
            #print(macenko_image.size)
            #print(vahadane_image.size)

            image_transformed1 = self.transform(macenko_image)
            #print(type(image_transformed1))

            image_transformed2 = self.transform(vahadane_image)
            #print("image size")
            #print(image_transformed1.size())
            #print(image_transformed2.size())


            if self.images[index].size != 150528:
              #print("kirrrrrrrrr")
              #print(self.images[index].shape)
              image_transformed1 = torch.zeros((3, 224, 224))
              image_transformed2 = torch.zeros((3, 224, 224))



            return image_transformed1, image_transformed2, label

    def __len__(self):
        return len(self.images)


In [17]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(degrees=(-45, 45)),  # Randomly rotate the image within the range of -45 to 45 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust brightness, contrast, saturation, and hue
    transforms.RandomVerticalFlip(),  # Randomly flip the image vertically
    #transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=(-10, 10)),  # Random affine transformation
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Random perspective transformation
    transforms.ToTensor(),  # Converts the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize the tensor with mean and standard deviation
])

In [18]:
# prompt: read the model from path

model = MyModel()
model.load_state_dict(torch.load('/content/drive/MyDrive/model_state.pt'))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [19]:
import torch
import torch.nn as nn
from torchvision import models


class MyModel2(nn.Module):
    def __init__(self, backbone1):
        super(MyModel2, self).__init__()
        # Use the pretrained ResNet18 model as the backbone
        self.backbone1 = backbone1
        for param in self.backbone1.parameters():
          param.requires_grad = False

        # Modify the first layer to accept 1 channel input
        #self.backbone1.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#
        ## Reduce the last layer of ResNet18 to 256 outputs
        #self.backbone1.fc = nn.Linear(self.backbone1.fc.in_features, 256)
#
        ## Define the first header with 3 layers
        #self.header1 = nn.Sequential(
        #    nn.Linear(256, 512),
        #    nn.ReLU(),
        #    nn.Linear(512, 512),
        #    nn.ReLU(),
        #    nn.Linear(512, 512),
        #)

        # Another instance of ResNet18 after the first header
        self.backbone2 = models.resnet18(pretrained=True)

        # Modify the first layer to accept 512 channels as the output of the first header
        self.backbone2.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # Reduce the last layer of the second ResNet18 to 256 outputs
        self.backbone2.fc = nn.Linear(self.backbone2.fc.in_features, 256)

        # Define the second header with 3 layers
        self.header2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
        )

    def forward(self, x):
        # Extract features from the first backbone
        features1 = self.backbone1(x)
        inputs1 = features1.view(features1.size(0), 1, features1.size(1), 1)

        # Pass the features through the first header
        #output1 = self.header1(features1)

        # Extract features from the second backbone using the output of the first header
        features2 = self.backbone2(inputs1)

        # Pass the features through the second header
        output2 = self.header2(features2)

        return output2




In [20]:
model.backbone.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [21]:
# Create an instance of the model
model2 = MyModel2(model.backbone)

In [22]:
# prompt: define device and use gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model2 = model2.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [23]:
model2 = model2.to(device)

In [29]:
temperature = .5
batch_size = 64
contrastive_loss = NT_Xent(batch_size, temperature, world_size=1)
#inter_cxr_contrastive = contrastive_loss(inter_cxr_outputs, inter_cxr_outputs_aug)


In [30]:
# Create an instance of CustomDataset
train_dataset = CustomDataset(train_images, train_labels, transform=transform)

#test_dataset = CustomDataset(test_images, test_labels, transform=transform)

# Create a DataLoader
batch_size = 64  # Set your desired batch size
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:

num_epochs = 1

for epoch in range(num_epochs):
    running_loss = 0.0

    # Iterate over the training data
    for i, data in enumerate(train_dataloader):
        # Get the inputs
        inputs1, inputs2, labels = data

        # Move data to the device
        inputs1 = inputs1.to(device)
        inputs2 = inputs2.to(device)
        #labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()
        #print(inputs1.device)
        #print(model.device)

        # Forward pass
        outputs1 = model2(inputs1)
        outputs2 = model2(inputs2)
        #if torch.all(inputs1 == 0):
        #
        #  outputs1 = torch.zeros_like(outputs1)
        #  outputs2 = torch.zeros_like(outputs2)
        for j in range(64):
                       # Check if the entire sample in cxr_data is all zero
            if torch.all(inputs1[j] == 0):
                print('hahahahahahahah')
                # Set the corresponding row in inter_ehr_outputs to zero
                outputs1[j] = 0
                outputs2[j] = 0

        #print(len(outputs1))
        #print(len(outputs2))
        #print(outputs1.shape)
        #print(outputs2.shape)
        #if torch.all(inputs1 == 0):
          #print("fuck")

        if len(outputs1) == 64 and len(outputs2) == 64:
          loss = contrastive_loss(outputs1, outputs2)

          # Backward pass
          loss.backward()

          # Update the parameters
          optimizer.step()

          # Print statistics
          running_loss += loss.item()
          if i % 1 == 0:  # Print every 2000 mini-batches
              print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}")
              running_loss = 0.0

print("Finished Training")


In [ ]:

image_datasets = {x: datasets.ImageFolder( dirs[x],   transform=data_transforms[x]) for x in ['train', 'test']}
# load the data into batches
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train',  'test']}
dataset_sizes = {x: len(image_datasets[x])
                              for x in ['train', 'test']}

In [ ]:
for epoch in range(10):
    running_train_loss = 0.0
    running_accuracy = 0.0
    running_vall_loss = 0.0
    total = 0
    ## model.eval() to model.train
    model1.train()
    torch.save(model1.state_dict(), os.path.join("/content/drive/MyDrive/save_train_simclr", 'epoch-{}.pth'.format(epoch)))
    for i, (images, labels) in enumerate(dataloaders['train']):
        #print("PASSED!!")
        # Forward pass
        images, labels = images.to(device), labels.to(device)
        outputs = model1(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
          print (f'Epoch : {epoch+1} , Step : {i+1} , Loss: {loss.item():.4f}')
